In [13]:
import time
from copy import deepcopy
import argparse
import sys
import traceback

import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
from torch.utils import data
from torch.utils.data import Dataset, DataLoader

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import gridspec
from utils import *


parser = argparse.ArgumentParser()
args = parser.parse_args("")
args.seed = 123
args.nbits = 2048
args.n_splits = 5
args.test_size = 0.2
args.num_mol = 50000
args.max_len = 120
args.shuffle = True
args.device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
args.optim = 'RMSProp'

np.random.seed(args.seed)
torch.manual_seed(args.seed)


# 1. Prepare Dataset

In [46]:
list_partition = make_partition(args)

# 2. Model Architecture

Create char_to_ix Ref: https://github.com/pytorch/tutorials/blob/master/beginner_source/nlp/word_embeddings_tutorial.py  
Pre-defined Embedding Layer Ref: https://medium.com/@martinpella/how-to-use-pre-trained-word-embeddings-in-pytorch-71ca59249f76

In [90]:
class Net(nn.Module):
        def __init__(self, vocab_size, args):
            super(Net, self).__init__()   
            self.embedding = self.create_emb_layer(vocab_size, args.emb_train)
            self.conv1 = nn.Conv2d(3, 6, 5)
            self.pool = nn.MaxPool2d(2, 2)
            self.conv2 = nn.Conv2d(6, 16, 5)
            self.fc1 = nn.Linear(16 * 5 * 5, 120)
            self.fc2 = nn.Linear(120, 84)
            self.fc3 = nn.Linear(84, 10)

        def forward(self, x):
            embeds = self.embedding(x)
            print(embeds.shape)
            x = self.pool(F.relu(self.conv1(x)))
            x = self.pool(F.relu(self.conv2(x)))
            x = x.view(-1, 16 * 5 * 5)
            x = F.relu(self.fc1(x))
            x = F.relu(self.fc2(x))
            x = self.fc3(x)
            return x
        
        def create_emb_layer(self, vocab_size, emb_train):
            emb_layer = nn.Embedding(vocab_size, vocab_size)
            weight_matrix = torch.zeros((vocab_size, vocab_size))
            for i in range(vocab_size):
                weight_matrix[i][i] = 1
            emb_layer.load_state_dict({'weight': weight_matrix})
            
            if not emb_train:
                emb_layer.weight.requires_grad = False
            return emb_layer

# 3. Train Model

In [91]:
def create_vocab(filename='../Data/logP/vocab.npy'):
    vocab = np.load(filename)
    vocab_size = len(vocab)
    char_to_ix = {char: i for i, char in enumerate(vocab)}
    return vocab, char_to_ix

In [102]:
def train(model, partition, optimizer, criterion, char_to_ix, args):
    data_iter = DataLoader(
        partition['train'],
        batch_size=args.batch_size,
        shuffle=args.shuffle
    )
    
    epoch_train_loss = 0
    for batch_idx, batch in enumerate(data_iter):
        X, y = batch[0], batch[1]
        X = torch.Tensor([[char_to_ix[c] for c in smile] for smile in X]).long()
        X, y = X.to(args.device), y.to(args.device)
    
        model.train()
        optimizer.zero_grad()

        pred_y = model(X)
        pred_y.require_grad = False
        train_loss = criterion(pred_y, y)
        epoch_train_loss += train_loss.item()
        train_loss.backward()
        optimizer.step()

        cnt_iter += 1

    return model, epoch_train_loss/cnt_iter

def experiment(partition, args):
    vocab, char_to_ix = create_vocab()
    model = Net(len(vocab), args)
    model.to(args.device)
    
    # Initialize Optimizer
    trainable_parameters = filter(lambda p: p.requires_grad, model.parameters())
    if args.optim == 'ADAM':
        optimizer = optim.Adam(trainable_parameters, lr=args.lr, weight_decay=args.l2_coef)
    elif args.optim == 'RMSProp':
        optimizer = optim.RMSprop(trainable_parameters, lr=args.lr, weight_decay=args.l2_coef)
    elif args.optim == 'SGD':
        optimizer = optim.SGD(trainable_parameters, lr=args.lr, weight_decay=args.l2_coef)
    else:
        assert False, "Undefined Optimizer Type"
        
    for epoch in range(args.epoch):
        model, train_loss = train(model, partition, optimizer, criterion, char_to_ix, args)
#         model, mse = test(model, partition, args)
#     return {'acc':acc, 'roc_auc':roc_auc, 'train_loss':train_loss, 'val_loss':val_loss, 'acc_test':acc_test, 'roc_auc_test':roc_auc_test}

In [103]:
args.exp_name = 'exp_hidden_dim'
args.input_dim = 2048
args.output_dim = 1
args.n_layer = 2
args.dropout_rate = 0.0
args.lr = 0.001
args.l2_coef = 0
args.optim = 'RMSProp'
args.epoch = 1
args.batch_size=128
args.emb_train = False
args.device = 'cuda' if torch.cuda.is_available() else 'cpu'

experiment(list_partition[0], args)

torch.Size([128, 120, 32])


RuntimeError: input has less dimensions than expected